In [679]:
from __future__ import division
from pyomo.environ import *
from pyomo.core.base.expr import identify_variables
import numpy as np

model = ConcreteModel()


model.n = range(6)

model.x = Var(model.n, within=Binary)

# model.c = np.array([1,2,2,1])
model.c = np.array([1,2,-1,3,-2,1])
# model.a = np.matrix([[-1,-1,0,0],[0,0,-1,-1]])
model.a = np.matrix([[-1,-4,1,0,0,0],[0,0,0,-3,-2,-1]])
# model.b = np.array([-1,-2])
model.b = np.array([-1,-2])
# model.eqmat = np.matrix([1,1,1,1])
model.eqmat = np.matrix([2,-5,1,-2,1,1])
# model.eqb = np.array([3])
model.eqb = np.array([0])
model.set = range(len(model.b))
consets = [[0],[1]]
# varsets = [[0,1],[2,3]]
varsets = [[0,1,2],[3,4,5]]
num_of_sets = len(consets)

In [680]:
def projectBinary(x):
    for row in range(len(x)):
        if x[row] <= 0.5:
            x[row] = 0
        else:
            x[row] = 1
    return x

def projectHyperplane(x,A,b):
    for row in range(A.shape[0]):
        if np.dot(A[row,:],x) - b[row] > 0:
            x = x - ((( (np.dot(np.array(A[row,:]),x))[0] - b[row] )/ ((np.array(np.square(A).sum(axis=1))[row])[0]) ) * np.array(A[row,:].flatten())[0])
    return x

def find_points(A,b):
    """Finds feasible points for a set of linear constraints"""
    points = []
    for i in range(1000):
        counter = 0
        x = np.random.rand(A.shape[1])
        x_old = x+1
        while (np.linalg.norm(x-x_old) > 0.01):
            if counter > 10:
                break
            x_old = x
            x = projectBinary(x)
            x = projectHyperplane(x,A,b)
            counter += 1
        if counter < 10:
            points.append(x)
        
    return np.unique(np.array(points),axis=0)


In [681]:
def create_sets(consets,varsets,Aineq,bineq):
    num_of_sets = len(consets)
    sets = {}
    lambda_sets = {}
    counter = 0
    for k in range(num_of_sets):
        set_k = {}
        set_k['constraint_rows'] = consets[k]
        set_k['indicies'] = varsets[k]
        set_k['points'] = find_points(Aineq[set_k['constraint_rows'],set_k['indicies']],bineq[set_k['constraint_rows']])
#         set_k['points'] = points1[k]
        set_k['num_points'] = len(set_k['points'])
        for count in range(set_k['num_points']):
            lambda_sets[counter] = (k,count)
            counter += 1
        sets[k] = set_k
    sets['num_sets'] = num_of_sets
    return sets,lambda_sets

points1 = [np.array([[1,1],[0,1]]),np.array([[1,1]])]

sets,lambda_sets = create_sets(consets,varsets,model.a,model.b)

In [626]:
find_points(np.matrix([[2,-5,1,-2,1,1],[-2,5,-1,2,-1,-1],[-1,-4,1,0,0,0],[0,0,0,-3,-2,-1]]),np.array([0,0,-1,-2]))

array([[1., 0., 0., 1., 0., 0.],
       [1., 1., 1., 0., 1., 1.]])

In [ ]:
points = [np.array([[0,1,0],[0,1,1],[1,0,0]]),np.array([[0,1,0],[0,1,1]])]

In [14]:
type(sets[0]['points'])

numpy.ndarray

In [682]:
def create_A_and_rhs(eqmat,eqb,sets):
    num_sets = sets['num_sets']
    num_eq = len(eqb)
    big_matrix = np.zeros((num_eq+num_sets,1))
    for k in range(num_sets):
        matrix_of_set_k = np.zeros((num_eq+num_sets,sets[k]['num_points']))
        # Now populate the matrix
        matrix_of_set_k[num_eq+k,:] = 1
        for j in range(num_eq):
            for column,point in enumerate(sets[k]['points']):
                matrix_of_set_k[j,column] = np.dot(eqmat[j,sets[k]['indicies']],point)[0,0]
        big_matrix = np.concatenate((big_matrix,matrix_of_set_k),axis=1)
    big_matrix = np.delete(big_matrix,0,axis=1)

    rhs = np.concatenate((model.eqb,np.ones(num_sets)))
    return big_matrix,rhs


A,rhs = create_A_and_rhs(model.eqmat,model.eqb,sets)
print(A,rhs)

(array([[-5., -4.,  2., -3., -2.,  1.,  2., -2., -1., -1.,  0.],
       [ 1.,  1.,  1.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  1.,  1.,  1.,  1.,  1.,  1.]]), array([0., 1., 1.]))


In [683]:
sets[0]['points']

array([[0., 1., 0.],
       [0., 1., 1.],
       [1., 0., 0.],
       [1., 1., 0.],
       [1., 1., 1.]])

In [684]:
def get_obj_values(c,sets):
    num_sets = sets['num_sets']
    obj_array = np.zeros(1)
    for k in range(num_sets):
        for point in sets[k]['points']:
            obj = np.dot(c[sets[k]['indicies']],point)
            obj_array = np.append(obj_array,obj)
    obj_array = np.delete(obj_array,0)
    return obj_array

print(get_obj_values(model.c,sets))

[ 2.  1.  1.  3.  2. -2. -1.  3.  4.  1.  2.]


In [685]:
def get_new_column_values(k,sets,point,eqmat,eqb,c):
    num_sets = sets['num_sets']
    num_eq = len(eqb)
    obj = np.dot(c[sets[k]['indicies']],point)
    Acol = np.zeros((num_eq+num_sets,1))
    Acol[k+num_eq,0] = 1
    for i in range(num_eq):
        Acol[i,0] = np.dot(eqmat[i,sets[k]['indicies']],point)[0,0]
    return obj,Acol

In [686]:
rmlp = ConcreteModel()
num_lambda = range(A.shape[1])
rmlp.n = num_lambda
print(rmlp.n)
rmlp.lambda_var = Var(rmlp.n,within=NonNegativeReals)
rmlp.con = ConstraintList()
A,rhs = create_A_and_rhs(model.eqmat,model.eqb,sets)
obj = get_obj_values(model.c,sets)
for i in range(A.shape[0]):
    rmlp.con.add(expr = sum( A[i,j]*rmlp.lambda_var[j] for j in rmlp.n ) == rhs[i])

rmlp.OBJ = Objective(expr = 
                    sum( obj[j]*rmlp.lambda_var[j] for j in rmlp.n ))

rmlp.dual = Suffix(direction=Suffix.IMPORT)
solver = SolverFactory('cplex',solver_io='nl')
solver.options['presolve'] = 0
solver.options['lpdisplay'] = 2
results = solver.solve(rmlp, tee = True)
rmlp.solutions.load_from(results)

convergenceflag = 0
while convergenceflag == 0:
    convergenceflag = 1
    print ("Duals")
    from pyomo.core import Constraint
    pi = np.zeros(len(model.eqb))
    mu = np.zeros(len(varsets))
    for c in rmlp.component_objects(Constraint, active=True):
        cobject = getattr(rmlp, str(c))
        for index in cobject:
            if index <= len(model.eqb):
                pi[index-1] = rmlp.dual[cobject[index]]
            else:
                mu[index-1-len(pi)] = rmlp.dual[cobject[index]]
            print ("      ", index, rmlp.dual[cobject[index]])
            
    # Now create the subproblem and use the dual values to solve
    num_sets = sets['num_sets']
    for k in range(num_sets):
        num_points = sets[k]['num_points']
        subprob = ConcreteModel()
        subprob.x = Var(range(len(sets[k]['indicies'])),within=Binary)
        subprob.con = ConstraintList()
        for set_num,i in enumerate(sets[k]['constraint_rows']):
            subprob.con.add(expr = 
                           sum(model.a[i,j]*subprob.x[col] for col,j in enumerate(sets[k]['indicies'])) <= model.b[i]
                           )
        constants = model.c[sets[k]['indicies']] - np.matmul(pi,np.array(model.eqmat[:,sets[k]['indicies']]))
#         constants = model.c[sets[k]['indicies']] - np.dot(pi,model.a[sets[k]['constraint_rows'],sets[k]['indicies']])[0,0]
        subprob.OBJ = Objective(expr = 
                               sum(constants[i]*subprob.x[i] for i in range(len(sets[k]['indicies']))) - mu[k]
                               )
        solver = SolverFactory('cplex')
        results = solver.solve(subprob, tee = True)
        subprob.solutions.load_from(results)
        if subprob.OBJ() < -10**-10:
            print("Does this print??")
            convergenceflag = 0
            print(subprob.OBJ())
            xval = []
            for v in subprob.component_objects(Var, active=True):
                for index in v:
                    xval.append(v[index].value)
            print(sets[k]['points'])
            print(np.array(xval))
            sets[k]['points'] = np.append(sets[k]['points'],[np.array(xval)],axis=0)
            lambda_sets[np.int(A.shape[1])] = (k,len(sets[k]['points'])-1)
            newc,newAcol = get_new_column_values(k,sets,xval,model.eqmat,model.eqb,model.c)
            obj = np.append(obj,newc)
            A = np.append(A,newAcol,axis=1)
            
    rmlp = ConcreteModel()
    
    num_lambda = range(A.shape[1])
    print(A)
    rmlp.n = num_lambda
    rmlp.lambda_var = Var(rmlp.n,within=NonNegativeReals)
    rmlp.con = ConstraintList()
    for i in range(A.shape[0]):
        rmlp.con.add(expr = sum( A[i,j]*rmlp.lambda_var[j] for j in rmlp.n ) == rhs[i])

    rmlp.OBJ = Objective(expr = 
                        sum( obj[j]*rmlp.lambda_var[j] for j in rmlp.n ))

    rmlp.dual = Suffix(direction=Suffix.IMPORT)
    solver = SolverFactory('cplex',solver_io='nl')
    solver.options['presolve'] = 0
    solver.options['lpdisplay'] = 2
    results = solver.solve(rmlp, tee = False)
    rmlp.solutions.load_from(results)
    
for v in rmlp.component_objects(Var, active=True):
    print("Variable component object",v)
    output = []
    for index in v:
        output.append(v[index].value)
        print("   ", index, v[index].value)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
CPLEX 12.8.0.0: presolve=0
lpdisplay=2
Initializing dual steep norms . . .
Duals
('      ', 1, 0.0)
('      ', 2, 1.0)
('      ', 3, -2.0)

Welcome to IBM(R) ILOG(R) CPLEX(R) Interactive Optimizer 12.8.0.0
  with Simplex, Mixed Integer & Barrier Optimizers
5725-A06 5725-A29 5724-Y48 5724-Y49 5724-Y54 5724-Y55 5655-Y21
Copyright IBM Corp. 1988, 2017.  All Rights Reserved.

Type 'help' for a list of available commands.
Type 'help' followed by a command name for more
information on commands.

CPLEX> Logfile 'cplex.log' closed.
Logfile 'c:\users\daniel~1\appdata\local\temp\tmpa16rcx.cplex.log' open.
CPLEX> Problem 'c:\users\daniel~1\appdata\local\temp\tmpgse_ag.pyomo.lp' read.
Read time = 0.00 sec. (0.00 ticks)
CPLEX> Problem name         : c:\users\daniel~1\appdata\local\temp\tmpgse_ag.pyomo.lp
Objective sense      : Minimize
Variables            :       4  [Nneg: 1,  Binary: 3]
Objective nonzeros   :       4
Linear constraints   :       2  [Less: 1,  Eq

In [687]:
def get_solution_values(output,sets,lambda_sets):
    solution = {}
    for index,i in enumerate(output):
        if i != 0:
            if lambda_sets[index][0] in solution:
                solution[lambda_sets[index][0]] += i*sets[lambda_sets[index][0]]['points'][lambda_sets[index][1]]
                
            else:
                solution[lambda_sets[index][0]] = i*sets[lambda_sets[index][0]]['points'][lambda_sets[index][1]]
            print("Set: {0}".format(lambda_sets[index][0]))
            print(i*sets[lambda_sets[index][0]]['points'][lambda_sets[index][1]])  
    return solution
solution = get_solution_values(output,sets,lambda_sets)
print(solution)

Set: 0
[0.  0.5 0.5]
Set: 0
[0.5 0.  0. ]
Set: 1
[0. 1. 0.]
{0: array([0.5, 0.5, 0.5]), 1: array([0., 1., 0.])}


In [196]:
sets[0]['lambda_indicies'][2]

3

In [650]:
rmlp = ConcreteModel()
num_lambda = range(A.shape[1])
rmlp.n = num_lambda
rmlp.lambda_var = Var(rmlp.n,within=Binary)
rmlp.con = ConstraintList()
for i in range(A.shape[0]):
    rmlp.con.add(expr = sum( A[i,j]*rmlp.lambda_var[j] for j in rmlp.n ) == rhs[i])

rmlp.OBJ = Objective(expr = 
                    sum( obj[j]*rmlp.lambda_var[j] for j in rmlp.n ))

rmlp.dual = Suffix(direction=Suffix.IMPORT)
solver = SolverFactory('cplex',solver_io='nl')
solver.options['presolve'] = 0
solver.options['lpdisplay'] = 2
results = solver.solve(rmlp, tee = True)
rmlp.solutions.load_from(results)

for v in rmlp.component_objects(Var, active=True):
    print("Variable component object",v)
    for index in v:
        print("   ", index, v[index].value)

CPLEX 12.8.0.0: presolve=0
lpdisplay=2
Using steepest-edge.
('Variable component object', <pyomo.core.base.var.IndexedVar object at 0x000000000A66C160>)
('   ', 0, 0.0)
('   ', 1, 1.0)
('   ', 2, 0.0)
('   ', 3, 1.0)
